In all versions 4.x the narrative gets cleaned as before and products are cleaned twice times. The first time half of the three most occuring entries in __Product__ are randomly removed. The second time , the components that almost never occur are removed. This run uses 10 different principle components. 

__Initialing the data__

In [2]:
import pandas as pd

df_selected1 = pd.read_csv("../balanced_data/corpus_balanced_cleaned_lemmatized_first_cut_scarce_elimination.csv", encoding="utf-8")

print("nulls in df_selected:", df_selected1["Consumer complaint narrative"].isnull().sum())
df_selected = df_selected1.dropna()

nulls in df_selected: 0


__Initializing the bag of words__

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

# vect (bag of words)
count_vect = CountVectorizer(
    stop_words="english",
    #ngram_range=(1,2), # bigrammen
    min_df=2, # only keep words that appear twice
    max_df=0.5 # appears max in 50% of documents
)

X_train_counts = count_vect.fit_transform(df_selected["Consumer complaint narrative"])

__Principal components are chosen__

In [4]:
from sklearn.decomposition import TruncatedSVD

tSVD = TruncatedSVD(n_components=10)

principal_components = tSVD.fit_transform(X_train_counts)
print(principal_components.shape)

(242494, 10)


In [5]:
print(principal_components)
principal_components_df = pd.DataFrame(principal_components)
print(principal_components_df)

[[ 4.26616402 -3.37081231  0.01908446 ...  0.54173743 -1.19513342
   0.37070827]
 [ 3.96354962 -1.099395    0.72606538 ...  0.16371173 -0.62118062
  -0.37312326]
 [14.47191872  7.74605066 -1.50423477 ... -1.66593713  1.42351616
   3.06942187]
 ...
 [ 0.7278814   0.60665109 -0.70427889 ...  0.1199834  -0.40595955
  -0.07083248]
 [ 5.77501698  4.62779867 -5.63698989 ... -0.84266606 -1.99947676
  -0.99502014]
 [ 6.52529338  1.60217494  0.5335259  ...  2.57380142 -2.09832668
   3.37020014]]
                0         1         2         3         4         5         6  \
0        4.266164 -3.370812  0.019084  1.007591  0.619244 -0.215616 -0.558814   
1        3.963550 -1.099395  0.726065 -0.560019  0.337041  2.802753 -0.221925   
2       14.471919  7.746051 -1.504235  1.584361  0.699492 -4.416701 -2.929847   
3        2.018344  0.900113  2.367420 -0.566462 -0.510036  1.861475  0.255470   
4        4.969348 -4.724663 -1.145122 -3.918889  3.566015  0.821757 -1.412371   
...           ...     

__One hot encoding of extra columns__

In [6]:
#One hot encoding
issue_df = pd.Categorical(df_selected['Issue'])

df_dummies = pd.get_dummies(issue_df, prefix = 'issue')

#X_train_counts_df = pd.DataFrame(X_train_counts)
#print("DF conversion done")

print(principal_components_df.shape)
print(df_dummies.shape)
df_concat = pd.concat([principal_components_df, df_dummies], axis = 1)
print(df_concat.shape)


(242494, 10)
(242494, 146)
(242494, 156)


__Split into train and test__

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df_concat, df_selected['Product'])
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(181870, 156)
(60624, 156)
(181870,)
(60624,)


__Application of linear SVM__

In [8]:
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report # do more stats

classifier = svm.LinearSVC()
classifier.fit(X_train, Y_train)

test_predictions = classifier.predict(X_test)
print(classification_report(test_predictions, Y_test))

C:\Users\l-Larsovic-l\anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


                                                    precision    recall  f1-score   support

                           Bank account or service       1.00      1.00      1.00      3702
                       Checking or savings account       0.99      1.00      1.00      4795
                                     Consumer Loan       0.95      0.79      0.87      2899
                                       Credit card       1.00      1.00      1.00      4667
                       Credit card or prepaid card       1.00      0.95      0.97      8268
                                  Credit reporting       1.00      1.00      1.00      7884
                                   Debt collection       1.00      1.00      1.00      9987
Money transfer, virtual currency, or money service       0.99      1.00      0.99      1864
                                          Mortgage       0.99      1.00      1.00      7580
         Payday loan, title loan, or personal loan       0.73      0.99      0.